## **TODO:** Set the value of `URL` to the URL from your learning materials

In [ ]:
URL = None
import os
assert URL and (type(URL) is str), "Be sure to initialize URL using the value from your learning materials"
os.environ['URL'] = URL

In [ ]:
%%bash
wget -q $URL -O ./data.zip
mkdir -p data
find *.zip | xargs unzip -o -d data/

## Use PyTorch `Dataset` and `Dataloader` with a structured dataset

In [ ]:
import os

import pandas as pd
import torch as pt

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

pt.set_default_dtype(pt.float64)

Read the files that match `part-*.csv` from the `data` subdirectory into a Pandas data frame named `df`.

In [ ]:
from pathlib import Path

df = pd.concat(
    pd.read_csv(file) for file in Path('data/').glob('part-*.csv')
)


## Explore the `df` data frame, including the column names, the first few rows of the dataset, and the data frame's memory usage.

In [ ]:
df[:5]

In [ ]:
df.info()

## Drop the `origindatetime_tr` column from the data frame. 

For now you are going to predict the taxi fare just based on the lat/lon coordinates of the pickup and the drop off locations. Remove the `origindatetime_tr` column from the data frame in your working dataset.

In [ ]:
working_df = df.drop('origindatetime_tr', axis = 1)
working_df.shape

## Sample 10% of your working dataset into a test dataset data frame

* **hint:** use the Pandas `sample` function with the dataframe. Specify a value for the `random_state` to achieve reproducibility.

In [ ]:
test_df = working_df.sample(frac = 0.10, random_state = 42)
test_df.shape

## Drop the rows that exist in your test dataset from the working dataset to produce a training dataset.

* **hint** DataFrame's `drop` function can use index values from a data frame to drop specific rows.

In [ ]:
train_df = working_df.drop(index = test_df.index)
train_df.shape

## Define 2 Python lists: 1st for the feature column names; 2nd for the target column name

In [ ]:
FEATURES = ['origin_block_latitude','origin_block_longitude','destination_block_latitude','destination_block_longitude']
TARGET = ['fareamount']

## Create `X` and `y` tensors with the values of your feature and target columns in the training dataset

In [ ]:
X = pt.tensor(train_df[FEATURES].values)
y = pt.tensor(train_df[TARGET].values)

## Create a `TensorDataset` instance with the `y` and `X` tensors (in that order)

In [ ]:
train_ds = TensorDataset(y, X)

## Create a `DataLoader` instance specifying a custom batch size

A batch size of `2 ** 18 = 262,144` should work well.

In [ ]:
BATCH_SIZE = 2 ** 18
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE)
len(train_dl)

## Create a model using `nn.Linear`

In [ ]:
w = nn.Linear(len(FEATURES), 1)


## Create an instance of the `AdamW` optimizer for the model

In [ ]:
optimizer = pt.optim.AdamW(w.parameters())

## Declare your `forward`, `loss` and `metric` functions

* **hint:** if you are tried of computing MSE by hand you can use `nn.functional.mse_loss` instead.

In [ ]:
def forward(X):
  return w(X)

def loss(y_pred, y):
  mse = nn.functional.mse_loss(y_pred, y)
  return mse, mse.sqrt()

## Iterate over the batches returned by your `DataLoader` instance

For every step of gradient descent, print out the MSE, RMSE, and the batch index
* **hint:** you can use Python's `enumerable` for an iterable
* **hint:** the batch returned by the `enumerable` has the same contents as your `TensorDataset` instance

In [ ]:
for batch_idx, batch in enumerate(train_dl):
  y, X = batch
  y_pred = forward(X)
  mse, rmse = loss(y_pred, y)
  mse.backward()
  print("Loss: ", mse.item(), " RMSE: ", rmse.item(), " Batch Idx: ", batch_idx)
  optimizer.step()
  optimizer.zero_grad()


## Implement 10 epochs of gradient descent training

For every step of gradient descent, printout the MSE, RMSE, epoch index, and batch index.

* **hint:** you can call `enumerate(DataLoader)` repeatedly in a `for` loop

In [ ]:
for epoch in range(10):
  for batch_idx, batch in enumerate(train_dl):
    y, X = batch
    y_pred = forward(X)
    mse, rmse = loss(y_pred, y)
    mse.backward()
    print(" Loss: ", mse.item(), " RMSE: ", rmse.item(), " Epoch: ", epoch, " Batch Idx: ", batch_idx)
    optimizer.step()
    optimizer.zero_grad()

Copyright 2021 CounterFactual.AI LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.